In [1]:
import requests
from bs4 import BeautifulSoup
import re
import ast
import csv

url = 'https://results.eci.gov.in/PcResultGenJune2024/index.htm#'

response = requests.get(url)
response.raise_for_status()

soup = BeautifulSoup(response.content, 'html.parser')

script_tag = soup.find('script', string=re.compile(r'var xValues ='))
if not script_tag:
    print("No relevant script tag found.")
    exit()

script_content = script_tag.string

x_values_pattern = re.compile(r'var xValues = (\[.*?\]);', re.DOTALL)
y_values_pattern = re.compile(r'var yValues = (\[.*?\]);', re.DOTALL)

x_values_str = x_values_pattern.search(script_content).group(1)
y_values_str = y_values_pattern.search(script_content).group(1)

x_values = ast.literal_eval(x_values_str)
y_values = ast.literal_eval(y_values_str)

paired_data = list(zip(x_values, y_values))

sorted_data = sorted(paired_data, key=lambda x: x[1], reverse=True)

highest_value_name = sorted_data[0][0]
highest_value = sorted_data[0][1]

with open('election_results.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Party', 'Seats'])
    writer.writerows(sorted_data)

print("Sorted Data:", sorted_data)
print("Highest Value Name:", highest_value_name)
print("Highest Value:", highest_value)


Sorted Data: [('BJP', 240), ('INC', 99), ('Others', 64), ('SP', 37), ('AITC', 29), ('DMK', 22), ('TDP', 16), ('JD(U)', 12), ('SHSUBT', 9), ('NCPSP', 8), ('SHS', 7)]
Highest Value Name: BJP
Highest Value: 240


In [3]:
#Vote Share Percentage
sum =0
for i in sorted_data:
    sum = sum + i[1]
print(sum)
for i in sorted_data:
    print(i[0],(i[1]/sum)*100,end="")
    print(" %")

543
BJP 44.19889502762431 %
INC 18.23204419889503 %
Others 11.786372007366483 %
SP 6.8139963167587485 %
AITC 5.3406998158379375 %
DMK 4.051565377532229 %
TDP 2.9465930018416207 %
JD(U) 2.209944751381215 %
SHSUBT 1.6574585635359116 %
NCPSP 1.4732965009208103 %
SHS 1.289134438305709 %


In [2]:
import requests
from bs4 import BeautifulSoup
import csv

# URL of the page to scrape
url = 'https://results.eci.gov.in/PcResultGenJune2024/index.htm#'

# Send a GET request to the page
response = requests.get(url)
response.raise_for_status()

# Parse the page content
soup = BeautifulSoup(response.content, 'html.parser')

# Find the select tag containing state options
select_tag = soup.find('select', {'id': 'ctl00_ContentPlaceHolder1_Result1_ddlState'})

# Extract state names and values
state_options = select_tag.find_all('option')
state_data = {option.text.strip(): option['value'] for option in state_options if option['value']}

# Print the extracted state data
print("State Data:", state_data)

# Base URL part for the new links
base_url = 'https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-'

# Dictionary to store merged data
merged_data = {}

# Loop through all states and extract details from each new URL
for state_name, state_value in state_data.items():
    new_url = f"{base_url}{state_value}.htm"
    new_response = requests.get(new_url)
    new_response.raise_for_status()
    new_soup = BeautifulSoup(new_response.content, 'html.parser')
    
    # Extract desired details from the new page
    rows = new_soup.find_all('div', class_='col-md-2 col-sm-6 col-12')
    state_results = []
    for row in rows:
        party_name = row.find('h4').text.strip()
        party_value = row.find('h2').text.strip()
        state_results.append((party_name, party_value))
    
    # Merge data under the same state name
    if state_name in merged_data:
        merged_data[state_name].extend(state_results)
    else:
        merged_data[state_name] = state_results

# Write the merged data to a CSV file
with open('election_results_by_state.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['State', 'Party', 'Seats'])
    for state, results in merged_data.items():
        for party, value in results:
            writer.writerow([state, party, value])

# Print the merged data
for state, results in merged_data.items():
    print(f"State: {state}")
    for party, value in results:
        print(f"  Party: {party}, Value: {value}")


State Data: {'Andaman & Nicobar Islands': 'U01', 'Andhra Pradesh': 'S01', 'Arunachal Pradesh': 'S02', 'Assam': 'S03', 'Bihar': 'S04', 'Chandigarh': 'U02', 'Chhattisgarh': 'S26', 'Dadra & Nagar Haveli and Daman & Diu': 'U03', 'Goa': 'S05', 'Gujarat': 'S06', 'Haryana': 'S07', 'Himachal Pradesh': 'S08', 'Jammu and Kashmir': 'U08', 'Jharkhand': 'S27', 'Karnataka': 'S10', 'Kerala': 'S11', 'Ladakh': 'U09', 'Lakshadweep': 'U06', 'Madhya Pradesh': 'S12', 'Maharashtra': 'S13', 'Manipur': 'S14', 'Meghalaya': 'S15', 'Mizoram': 'S16', 'Nagaland': 'S17', 'NCT OF Delhi': 'U05', 'Odisha': 'S18', 'Puducherry': 'U07', 'Punjab': 'S19', 'Rajasthan': 'S20', 'Sikkim': 'S21', 'Tamil Nadu': 'S22', 'Telangana': 'S29', 'Tripura': 'S23', 'Uttar Pradesh': 'S24', 'Uttarakhand': 'S28', 'West Bengal': 'S25'}
State: Andaman & Nicobar Islands
  Party: BJP, Value: 1
State: Andhra Pradesh
  Party: TDP, Value: 16
  Party: YSRCP, Value: 4
  Party: BJP, Value: 3
  Party: JnP, Value: 2
State: Arunachal Pradesh
  Party: BJP

In [4]:
import requests
from bs4 import BeautifulSoup
import csv

# Base URL for candidate wise result
base_url = 'https://results.eci.gov.in/PcResultGenJune2024/candidateswise-S01'

# State values extracted earlier
state_values = ['1', '2', '3', '4']

# Create a list to store all candidate data
all_candidate_data = []

# Loop through each state value
for state_value in state_values:
    # Construct the full URL
    url = f'{base_url}{state_value}.htm'

    # Send a GET request to the page
    response = requests.get(url)
    response.raise_for_status()

    # Parse the page content
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all candidate boxes
    candidate_boxes = soup.find_all('div', class_='col-md-4 col-12')

    # Print state name
    state_name = soup.find('title').text.strip()

    # Extract and print details for each candidate
    for box in candidate_boxes:
        candidate_name = box.find('h5').text.strip()
        party_name = box.find('h6').text.strip()
        status = box.find('div', class_='status').text.strip().capitalize()
        votes = box.find('div', class_='status').find_next_sibling('div').text.strip()
        image_src = box.find('figure').find('img')['src']
        candidate_id = image_src.split('/')[-1].split('.')[0] if image_src != 'img/nota.jpg' else 'NOTA'

        # Append candidate data to the list
        all_candidate_data.append([state_name, candidate_name, party_name, status, votes, candidate_id])

# Write all candidate data to a CSV file
with open('candidates_results.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['State', 'Candidate Name', 'Party', 'Status', 'Votes', 'Candidate ID'])
    writer.writerows(all_candidate_data)

# Print the data to verify
for data in all_candidate_data:
    print(data)


['Election Commission of India', 'GUMMA THANUJA RANI', 'Yuvajana Sramika Rythu Congress Party', 'Won\n477005 (+ 50580)', 'GUMMA THANUJA RANI\nYuvajana Sramika Rythu Congress Party', 'GUMMA-2024-5992']
['Election Commission of India', 'KOTHAPALLI GEETHA', 'Bharatiya Janata Party', 'Lost\n426425 ( -50580)', 'KOTHAPALLI GEETHA\nBharatiya Janata Party', 'KOTHA-2024-6969']
['Election Commission of India', 'APPALANARASA PACHIPENTA', 'Communist Party of India  (Marxist)', 'Lost\n123129 ( -353876)', 'APPALANARASA PACHIPENTA\nCommunist Party of India  (Marxist)', 'APPAL-2024-5751']
['Election Commission of India', 'AVASHYA LAHARI . VARAM', 'Bahujan Samaj Party', 'Lost\n25750 ( -451255)', 'AVASHYA LAHARI . VARAM\nBahujan Samaj Party', 'VARAM-2024-6929']
['Election Commission of India', 'SAMAREDDY BALAKRISHNA', 'Independent', 'Lost\n9535 ( -467470)', 'SAMAREDDY BALAKRISHNA\nIndependent', 'SAMAR-2024-7717']
['Election Commission of India', 'NIMMAKA JAYARAJU', 'Independent', 'Lost\n9462 ( -467543)'

In [9]:
import requests
from bs4 import BeautifulSoup
import csv

# Base URL for candidate wise result
base_url = 'https://results.eci.gov.in/PcResultGenJune2024/candidateswise-S01'

# State values extracted earlier
state_values = ['1', '2', '3', '4', '5']  # First 5 state values

# Create a list to store all candidate data
all_candidate_data = []

# Loop through each state value
for state_value in state_values:
    # Construct the full URL
    url = f'{base_url}{state_value}.htm'

    # Send a GET request to the page
    response = requests.get(url)
    response.raise_for_status()

    # Parse the page content
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all candidate boxes
    candidate_boxes = soup.find_all('div', class_='col-md-4 col-12')

    # Get state name
    state_name = soup.find('title').text.strip()

    # Extract details for each candidate
    for box in candidate_boxes:
        candidate_name = box.find('h5').text.strip()
        party_name = box.find('h6').text.strip()
        status = box.find('div', class_='status').text.strip().capitalize()
        votes = box.find('div', class_='status').find_next_sibling('div').text.strip()
        image_src = box.find('figure').find('img')['src']
        candidate_id = image_src.split('/')[-1].split('.')[0] if image_src != 'img/nota.jpg' else 'NOTA'

        # Append candidate data to the list
        all_candidate_data.append([state_name, candidate_name, party_name, status, votes, candidate_id])

    # Check if the "Next" link exists
    next_link = soup.find('a', string='Next')
    if next_link:
        next_base_url = next_link['href']
        base_url = f'https://results.eci.gov.in/PcResultGenJune2024/{next_base_url}'
    else:
        # If there's no "Next" link, stop the loop
        break

# Write all candidate data to a CSV file
with open('candidates_results.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['State', 'Candidate Name', 'Party', 'Status', 'Votes', 'Candidate ID'])
    writer.writerows(all_candidate_data)

# Print the data to verify
for data in all_candidate_data:
    print(data)


['Election Commission of India', 'GUMMA THANUJA RANI', 'Yuvajana Sramika Rythu Congress Party', 'Won\n477005 (+ 50580)', 'GUMMA THANUJA RANI\nYuvajana Sramika Rythu Congress Party', 'GUMMA-2024-5992']
['Election Commission of India', 'KOTHAPALLI GEETHA', 'Bharatiya Janata Party', 'Lost\n426425 ( -50580)', 'KOTHAPALLI GEETHA\nBharatiya Janata Party', 'KOTHA-2024-6969']
['Election Commission of India', 'APPALANARASA PACHIPENTA', 'Communist Party of India  (Marxist)', 'Lost\n123129 ( -353876)', 'APPALANARASA PACHIPENTA\nCommunist Party of India  (Marxist)', 'APPAL-2024-5751']
['Election Commission of India', 'AVASHYA LAHARI . VARAM', 'Bahujan Samaj Party', 'Lost\n25750 ( -451255)', 'AVASHYA LAHARI . VARAM\nBahujan Samaj Party', 'VARAM-2024-6929']
['Election Commission of India', 'SAMAREDDY BALAKRISHNA', 'Independent', 'Lost\n9535 ( -467470)', 'SAMAREDDY BALAKRISHNA\nIndependent', 'SAMAR-2024-7717']
['Election Commission of India', 'NIMMAKA JAYARAJU', 'Independent', 'Lost\n9462 ( -467543)'